<a href="https://colab.research.google.com/github/ttogle918/ds-section2-sprint3/blob/main/n232-data-wrangling/n232a-data-wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *AIB / SECTION 2 / SPRINT 3 / NOTE 2*

# 📝 Assignment
---
# Data Wrangling


### 선택한 데이터 wrangling

In [1]:
# google 드라이브에 로컬 파일 업로드
from google.colab import files, drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
credits = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/code_states/credits.csv")
keywords = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/code_states/keywords.csv")
links = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/code_states/links.csv")
links_small = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/code_states/links_small.csv")
movies_metadata = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/code_states/movies_metadata.csv")
ratings = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/code_states/ratings.csv")
ratings_small = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/code_states/ratings_small.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
import pandas as pd
import numpy as np
import ast
import re
from collections import Counter

In [4]:
import ast
def getDataFrame(s) :
  if s[0] == '\'' :
    s = s[1:-1]
  if str(s)[0] == '{' :
    df = pd.DataFrame([ast.literal_eval(s)])
  if str(s)[0] == '[' :
    df = pd.DataFrame(ast.literal_eval(s))
  else :
    df = s
    print(s)
  return df

In [5]:
print(credits.shape)
credits.head(1)

(45476, 3)


,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862


In [6]:
len(credits['id'].unique()) # id당 한 row가 아니다!

45432

In [7]:
print(keywords.shape)
keywords.head(1)

(46419, 2)


,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."


In [8]:
print(links.shape)
links.head(1)

(45843, 3)


,movieId,imdbId,tmdbId
0,1,114709,862.0


In [9]:
print(links_small.shape)
links_small.head(1)

(9125, 3)


,movieId,imdbId,tmdbId
0,1,114709,862.0


In [10]:
print(movies_metadata.shape)
movies_metadata.head(1)

(45466, 24)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [11]:
print(ratings.shape)
ratings.head(1)

(26024289, 4)


,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529


In [12]:
print(ratings_small.shape)
ratings_small.head(1)

(100004, 4)


,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144


### credits : 극 중 등장인물에 대한 정보와 제작진에 대한 정보를 담고있는 dataframe

In [13]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [14]:
# 1. 중복된 값 지우기
credits.duplicated().sum()

37

In [15]:
credits = credits.drop_duplicates()

In [16]:
print('empty cast', len(credits[credits.cast == '[]']) )
print('empty crew', len(credits[credits.crew == '[]']) )

empty cast 2414
empty crew 771


In [17]:
# 2. null값, empty값 확인 -> 지우기
credits = credits[(credits.cast != '[]') & (credits.crew != '[]')]
credits.shape

(42675, 3)

In [18]:
print(len(credits['id'].unique())) # id당 한 row가 아니다!

42668


In [19]:
# cast의 내용은 같고 credit_id의 값만 다르기 때문에 삭제해도 된다 -> id는 unique()
credits = credits.drop(credits[credits['id'].duplicated() == True].index)
print(credits.shape)
credits.head(3)

(42668, 3)


,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602


### cast

In [20]:
# 4. cast에서 배우 이름 따로 저장, crew에서 감독 이름 따로 저장
# string 형태로 data 형태 바꾸기
li = [entry['name'] for entry in  ast.literal_eval(credits.cast[0])[0:4]]
li

['Tom Hanks', 'Tim Allen', 'Don Rickles', 'Jim Varney']

In [21]:
# cast 상위 8명만
import re

def set_list_to_string(s, idx, to) :
  if to == 0 :
    t_dict = ast.literal_eval(s)
    return [re.sub(r'[^\w]', '_',re.sub(r'[^\w\s]', '', entry[idx])) for entry in t_dict]
  
  t_dict = ast.literal_eval(s)[0:to]
  return [re.sub(r'[^\w]', '_',re.sub(r'[^\w\s]', '', entry[idx])) for entry in t_dict]

In [22]:
credits['actors'] = credits.cast.apply(set_list_to_string, idx='name', to=8)
credits['actors'].head(10)

0    [Tom_Hanks, Tim_Allen, Don_Rickles, Jim_Varney...
1    [Robin_Williams, Jonathan_Hyde, Kirsten_Dunst,...
2    [Walter_Matthau, Jack_Lemmon, AnnMargret, Soph...
3    [Whitney_Houston, Angela_Bassett, Loretta_Devi...
4    [Steve_Martin, Diane_Keaton, Martin_Short, Kim...
5    [Al_Pacino, Robert_De_Niro, Val_Kilmer, Jon_Vo...
6    [Harrison_Ford, Julia_Ormond, Greg_Kinnear, An...
7    [Jonathan_Taylor_Thomas, Brad_Renfro, Rachael_...
8    [JeanClaude_Van_Damme, Powers_Boothe, Dorian_H...
9    [Pierce_Brosnan, Sean_Bean, Izabella_Scorupco,...
Name: actors, dtype: object

In [23]:
# 가장 많이 나온 배우 40명을 most_popular_acctors이라고 가정
from collections import Counter

actor_count = Counter([name for names_per_film in credits.actors.values for name in names_per_film])
most_popular_actors = [entry[0] for entry in actor_count.most_common(40)]
credits['number_of_named_actors'] = credits['actors'].apply(len)
credits.head(3)

,cast,crew,id,actors,number_of_named_actors
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,"[Tom_Hanks, Tim_Allen, Don_Rickles, Jim_Varney...",8
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,"[Robin_Williams, Jonathan_Hyde, Kirsten_Dunst,...",8
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,"[Walter_Matthau, Jack_Lemmon, AnnMargret, Soph...",7


In [24]:
def listToString(item_list):
  return r' '.join(item_list)

credits['actors'] = credits['actors'].apply(listToString)

### Crew

In [25]:
credits['crew'] = credits.crew.apply(set_list_to_string, idx='name', to=4)
credits.crew[0]

['John_Lasseter', 'Joss_Whedon', 'Andrew_Stanton', 'Joel_Cohen']

In [26]:
# 가장 많이 나온 crew 40명을 most_popular_crew라고 가정
from collections import Counter

crew_count = Counter([name for names_per_film in credits.crew.values for name in names_per_film])
most_popular_crews = [entry[0] for entry in crew_count.most_common(40)]
credits['number_of_named_crews'] = credits['crew'].apply(len)

credits['crew'] = credits['crew'].apply(listToString)
credits.head(3)

,cast,crew,id,actors,number_of_named_actors,number_of_named_crews
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...",John_Lasseter Joss_Whedon Andrew_Stanton Joel_...,862,Tom_Hanks Tim_Allen Don_Rickles Jim_Varney Wal...,8,4
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...",Larry_J_Franco Jonathan_Hensleigh James_Horner...,8844,Robin_Williams Jonathan_Hyde Kirsten_Dunst Bra...,8,4
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...",Howard_Deutch Mark_Steven_Johnson Mark_Steven_...,15602,Walter_Matthau Jack_Lemmon AnnMargret Sophia_L...,7,4


In [27]:
credits = credits.drop(['cast'], axis=1)

### 배우들이 영화에 주연으로 출연한 수에 따른 그래프와 영화에 참여한 크루의 수

In [28]:
x_value = list()
y_value = list()
for i in range(40) :
  x_value.append(actor_count.most_common(40)[i][0])
  y_value.append(actor_count.most_common(40)[i][1])

df_actor_count = pd.DataFrame(data={'actor':x_value, 'count':y_value})
df_actor_count.head(10)

,actor,count
0,Christopher_Lee,115
1,John_Wayne,112
2,Jackie_Chan,103
3,Gérard_Depardieu,103
4,Michael_Caine,102
5,Robert_De_Niro,100
6,Samuel_L_Jackson,97
7,Donald_Sutherland,92
8,Danny_Glover,86
9,Harvey_Keitel,85


In [29]:
x_value = list()
y_value = list()
for i in range(40) :
  x_value.append(crew_count.most_common(40)[i][0])
  y_value.append(crew_count.most_common(40)[i][1])

df_crew_count = pd.DataFrame(data={'crew':x_value, 'count':y_value})
df_crew_count.head(10)

,crew,count
0,Cedric_Gibbons,189
1,Avy_Kaufman,132
2,Roger_Corman,127
3,Max_Steiner,122
4,Ennio_Morricone,116
5,Darryl_F_Zanuck,116
6,Charlie_Chaplin,115
7,Luc_Besson,108
8,Werner_Herzog,104
9,Hal_B_Wallis,102


In [30]:
credits.head(3)

,crew,id,actors,number_of_named_actors,number_of_named_crews
0,John_Lasseter Joss_Whedon Andrew_Stanton Joel_...,862,Tom_Hanks Tim_Allen Don_Rickles Jim_Varney Wal...,8,4
1,Larry_J_Franco Jonathan_Hensleigh James_Horner...,8844,Robin_Williams Jonathan_Hyde Kirsten_Dunst Bra...,8,4
2,Howard_Deutch Mark_Steven_Johnson Mark_Steven_...,15602,Walter_Matthau Jack_Lemmon AnnMargret Sophia_L...,7,4


In [31]:
merged_df = pd.merge(credits, keywords, how='inner', on='id')

### links : dataframe끼리 연결해주는 id를 가지는 데이터프레임

In [32]:
links.columns = ['movieId', 'imdbId', 'id']
links = links.drop_duplicates()
links.isnull().sum()

movieId      0
imdbId       0
id         219
dtype: int64

In [33]:
merged_df = pd.merge(merged_df, links, how='inner', on='id')
merged_df.head(3)

,crew,id,actors,number_of_named_actors,number_of_named_crews,keywords,movieId,imdbId
0,John_Lasseter Joss_Whedon Andrew_Stanton Joel_...,862,Tom_Hanks Tim_Allen Don_Rickles Jim_Varney Wal...,8,4,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...",1,114709
1,Larry_J_Franco Jonathan_Hensleigh James_Horner...,8844,Robin_Williams Jonathan_Hyde Kirsten_Dunst Bra...,8,4,"[{'id': 10090, 'name': 'board game'}, {'id': 1...",2,113497
2,Howard_Deutch Mark_Steven_Johnson Mark_Steven_...,15602,Walter_Matthau Jack_Lemmon AnnMargret Sophia_L...,7,4,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392...",3,113228


### movies_metadata : 영화에 대한 정보를 담고있는 dataframe

In [34]:
movies_metadata.duplicated().sum()

13

In [35]:
movies_metadata = movies_metadata.drop_duplicates()
print(movies_metadata.shape)
movies_metadata.head(2)

(45453, 24)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [36]:
print(movies_metadata.revenue.mean())
print(movies_metadata.revenue.median())
movies_metadata.status.unique()

11212544.046889784
0.0


array(['Released', nan, 'Rumored', 'Post Production', 'In Production',
       'Planned', 'Canceled'], dtype=object)

In [37]:
movies_metadata.isnull().sum()

adult                        0
belongs_to_collection    40959
budget                       0
genres                       0
homepage                 37673
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25045
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [38]:
def toint(s) :
  try :
    n = int(s)
    return n
  except :
    print(s)
    return s
movies_metadata.budget.apply(toint)

/ff9qCepilowshEtG2GYWwzt2bs4.jpg
/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg
/zaSf5OG7V8X8gqFvly88zDdRm46.jpg


0        30000000
1        65000000
2               0
3        16000000
4               0
           ...   
45461           0
45462           0
45463           0
45464           0
45465           0
Name: budget, Length: 45453, dtype: object

In [39]:
# 본인의 columns 위치가 아닌 곳에 데이터가 있는 data들 drop
movies_metadata = movies_metadata.drop(index=movies_metadata[movies_metadata.budget == '/ff9qCepilowshEtG2GYWwzt2bs4.jpg'].index)
movies_metadata = movies_metadata.drop(index=movies_metadata[movies_metadata.budget == '/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg'].index)
movies_metadata = movies_metadata.drop(index=movies_metadata[movies_metadata.budget == '/zaSf5OG7V8X8gqFvly88zDdRm46.jpg'].index)
print(movies_metadata.shape)
movies_metadata.id = movies_metadata.id.astype(int)

(45450, 24)


In [40]:
# nan 값을 0으로 치환(없다고 가정한다)
movies_metadata.poster_path = movies_metadata.poster_path.fillna(0)

def set_poster(s) :
  if s == 0 :
    return 0
  else :
    return 1
  return 0
movies_metadata.poster_path = movies_metadata.poster_path.apply(set_poster)
movies_metadata.columns = ['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count']
movies_metadata.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster', 'production_companies', 'production_countries',
       'release_date', 'revenue', 'runtime', 'spoken_languages', 'status',
       'tagline', 'title', 'video', 'vote_average', 'vote_count'],
      dtype='object')

In [41]:
print(movies_metadata.adult.unique())
print(movies_metadata.status.unique())
print(movies_metadata.video.unique())

['False' 'True']
['Released' nan 'Rumored' 'Post Production' 'In Production' 'Planned'
 'Canceled']
[False True nan]


In [42]:
# False가 대부분이고, True는 극소수(대략 0.02%)이므로 통계적으로 의미를 띄지않는다.(유의미하지않다.) -> drop
Counter(movies_metadata.adult)

Counter({'False': 45441, 'True': 9})

In [43]:
# belongs_to_collection 컬럼에 빈칸(nan)값이 많기 때문에 우리가 예측하고자 하는 바에 도움이 되지 않는다. -> drop
movies_metadata.shape, movies_metadata.belongs_to_collection.isnull().sum()

((45450, 24), 40959)

In [44]:
# dic형식으로 된 내용을 name컬럼만 빼서 list로 저장
movies_metadata['genres'] = movies_metadata.genres.apply(set_list_to_string, idx='name', to=0)
movies_metadata['number_of_popular_genres'] = movies_metadata['genres'].apply(len)

In [45]:
from collections import Counter

genre_count = Counter([name for names_per_film in movies_metadata.genres.values for name in names_per_film])
most_popular_genres = [entry[0] for entry in genre_count.most_common(40)]

movies_metadata['genres'] = movies_metadata['genres'].apply(listToString)

In [46]:
print(movies_metadata.shape)
movies_metadata = movies_metadata.drop(movies_metadata[movies_metadata['production_companies'].isnull()==True].index)
print(movies_metadata.shape)

(45450, 25)
(45447, 25)


In [47]:
# dic형식으로 된 내용을 name컬럼만 빼서 list로 저장
movies_metadata['production_companies'] = movies_metadata.production_companies.apply(set_list_to_string, idx='name', to=0)
movies_metadata['number_of_production_companies'] = movies_metadata['production_companies'].apply(len)

# 가장 많이 나온 genres 40명을 most_popular_genres라고 가정
companies_count = Counter([name for names_per_film in movies_metadata.production_companies.values for name in names_per_film])
most_popular_companies = [entry[0] for entry in companies_count.most_common(40)]

movies_metadata['production_companies'] = movies_metadata['production_companies'].apply(listToString)

In [48]:
# dic형식으로 된 내용을 name컬럼만 빼서 list로 저장
movies_metadata['production_countries'] = movies_metadata.production_countries.apply(set_list_to_string, idx='name', to=0)
movies_metadata['number_of_production_countries'] = movies_metadata['production_countries'].apply(len)

# 가장 많이 나온 genres 40명을 most_popular_genres라고 가정
countries_count = Counter([name for names_per_film in movies_metadata.production_countries.values for name in names_per_film])
most_popular_countries = [entry[0] for entry in countries_count.most_common(40)]

movies_metadata['production_countries'] = movies_metadata['production_countries'].apply(listToString)

movies_metadata = movies_metadata.dropna()

In [49]:
# 홈페이지가 존재한다면 1, 없다면 0으로 치환
def setHomepage(s) :
  if s == 0 :
    return 0
  s = str(s)
  if s[0:4] == 'http' :
    return 1
  return 0
movies_metadata.fillna(0) # homepage가 없는 경우 0
movies_metadata.homepage = movies_metadata.homepage.apply(setHomepage)


# 음성언어가 몇가지 있는지 치환
movies_metadata.spoken_languages = movies_metadata.spoken_languages.apply(set_list_to_string, idx='name', to=0)
movies_metadata.spoken_languages = movies_metadata.spoken_languages.apply(len)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [50]:
# 출시된 상태의 영화에 대해서 훈련시킬 예정이다.
movies_metadata = movies_metadata[ (movies_metadata['status'] == 'Released') | (movies_metadata['status'] == 'Planned') 
                    | (movies_metadata['status'] == 'In Production') | (movies_metadata['status'] == 'Post Production')]
print(movies_metadata.status.unique())

['Released']


### 무의미한 데이터 삭제

In [51]:
movies_metadata = movies_metadata.drop(['adult', 'poster', 'belongs_to_collection', 'imdb_id', 'vote_count', 
                                        'original_language', 'original_title', 'overview', 'tagline'], axis=1)
movies_metadata['id'] = movies_metadata['id'].astype(int)
movies_metadata = movies_metadata.drop( movies_metadata[movies_metadata['runtime'] < 60 ].index)
merged_df = pd.merge(merged_df, movies_metadata, how='inner', on='id')

### ratings : 영화에 대한 평점을 담고있는 dataframe

In [52]:
ratings = ratings.drop_duplicates(ignore_index=True)  # 중복값 없음
ratings = ratings.drop(['userId', 'timestamp'], axis=1)
grouped_rating = (ratings.groupby(by="movieId").mean())['rating'].to_frame()
grouped_rating['rating'] = grouped_rating['rating'].round(1)
grouped_rating = grouped_rating.reset_index()

In [53]:
def transform_rating(n) :
  if n >= 3.5 :
    return True
  return False

grouped_rating['rating'] = grouped_rating['rating'].apply(transform_rating)

In [54]:
merged_df = pd.merge(merged_df, grouped_rating, how='inner', on='movieId')
merged_df.head(3)

,crew,id,actors,number_of_named_actors,number_of_named_crews,keywords,movieId,imdbId,budget,genres,homepage,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,title,video,vote_average,number_of_popular_genres,number_of_production_companies,number_of_production_countries,rating
0,Martin_Campbell Ian_Fleming Jeffrey_Caine Bruc...,710,Pierce_Brosnan Sean_Bean Izabella_Scorupco Fam...,8,4,"[{'id': 701, 'name': 'cuba'}, {'id': 769, 'nam...",10,113189,58000000,Adventure Action Thriller,1,14.686,United_Artists Eon_Productions,United_Kingdom United_States_of_America,1995-11-16,352194034.0,130.0,3,Released,GoldenEye,False,6.6,3,2,2,False
1,F_Gary_Gray DJ_Pooh Ice_Cube Frank_Fitzpatrick,10634,Ice_Cube Chris_Tucker Nia_Long Tom_Lister_Jr J...,8,4,"[{'id': 897, 'name': 'rap music'}, {'id': 970,...",69,113118,3500000,Comedy,1,14.5696,New_Line_Cinema,United_States_of_America,1995-04-26,28215918.0,91.0,1,Released,Friday,False,7.0,1,1,1,True
2,Robert_Rodriguez Quentin_Tarantino Quentin_Tar...,755,George_Clooney Quentin_Tarantino Harvey_Keitel...,8,4,"[{'id': 246, 'name': 'dancing'}, {'id': 380, '...",70,116367,19000000,Horror Action Thriller Crime,1,15.3392,A_Band_Apart Dimension_Films Los_Hooligans_Pro...,United_States_of_America,1996-01-19,25836616.0,108.0,2,Released,From Dusk Till Dawn,False,6.9,4,4,1,False


### 1) 지속적으로 여러분의 포트폴리오 데이터를 랭글링 하세요
- 여러분이 선택한 데이터들을 랭글링하여 모델을 만들기 위한 테이블 형태의 데이터로 만들어 보세요.
- **다음 함수의 result를 완성시키고 리턴값을 제출하세요.**

In [55]:
import pandas as pd
df = pd.read_csv("https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/assignments/purchase_data.csv", sep=",",index_col=0)

## 다음 result 함수 안에 male, female 변수를 계산하세요.(groupby, get_group을 사용하세요.)
## sum_male: Age = 20, Gender = "Male" 그룹의 Price를 합하세요.
## sum_female: Age = 20, Gender = "Female" 그룹의 Price를 합하세요.

df.head()

,SN,Age,Gender,Item ID,Item Name,Price
Purchase ID,,,,,,
0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,Ithergue48,24,Male,92,Final Critic,4.88
3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,Iskosia90,23,Male,131,Fury,1.44


In [56]:
df.dtypes

SN            object
Age            int64
Gender        object
Item ID        int64
Item Name     object
Price        float64
dtype: object

In [57]:
def result(df):
    cond = (df.Age==20) & (df.Gender == 'Male')
    sum_male = df[(df.Age==20) & (df.Gender == 'Male')].Price.sum()
    sum_female = df[(df.Age==20) & (df.Gender == 'Female')].Price.sum()

    print("sum_male: {}, sum_female: {}".format(sum_male, sum_female))
    return round(sum_male + sum_female)

result(df)

sum_male: 260.2, sum_female: 43.730000000000004


304

### 2) 새로 특징들을 만들어 보세요
- 여러분이 선택한 데이터에서 특징 공학을 수행해 보세요.
- **(다중선택) purchase_data를 랭글링하여 다음 조건을 계산하고 결과가 올바른 항목을 세요.**
    1. 나이 1-20세 그룹(20세 포함)의 평균 Price는 3.101818이다.
    2. 나이 21-50세 그룹의 평균 Price는 3.021559이다.
    3. 가장 많이 구입한 아이템의 ID는 178 이다.
    4. 두번째로 매출을 많이 낸 아이템 ID는 145이다.

In [58]:
### 이곳에서 제출폼 과제를 진행해 주세요 ###
# 그룹별 Price 평균
df_grouped_by_age = pd.DataFrame(df.groupby('Age').sum('Price').Price)
df_count = df.groupby('Age').count().SN.to_frame()
df_count.columns = ['num']
print("가장 높은 나이 : ", df_grouped_by_age.index.max())


df_merge = df_grouped_by_age.merge(df_count, on='Age', how='inner')
df_merge_sum =df_merge[df_merge.index <= 20].num.sum()

print("나이 1-20세 그룹(20세 포함)의 평균 Price", sum(df_grouped_by_age[df_grouped_by_age.index <= 20 ].Price)/df_merge_sum )

df_merge_sum =df_merge[df_merge.index > 20].num.sum()
print("나이 21-50세 그룹의 평균 Price", sum(df_grouped_by_age[20 < df_grouped_by_age.index ].Price)/df_merge_sum )


가장 높은 나이 :  45
나이 1-20세 그룹(20세 포함)의 평균 Price 3.1018181818181816
나이 21-50세 그룹의 평균 Price 3.0215587044534415


In [59]:
df_merge.head()

,Price,num
Age,,
7,32.89,9
8,25.97,8
9,18.27,6
10,31.83,9
11,18.79,7


In [60]:
df_count.head()

,num
Age,
7,9
8,8
9,6
10,9
11,7


In [61]:
# 판매수 상위 Item ID
df_grouped_by_id = pd.DataFrame(df.groupby('Item ID').count().SN)
df_grouped_by_id.sort_values('SN', ascending=False)

,SN
Item ID,
178,12
145,9
108,9
82,9
19,8
...,...
104,1
23,1
180,1


In [62]:
# print("가장 많이 구입한 아이템의 ID는", df_grouped_by_id.sort_values('SN', ascending=False)[:1].SN)

print("가장 많이 구입한 아이템의 ID는", df_grouped_by_id[df_grouped_by_id.SN == df_grouped_by_id.SN.max()].index)

가장 많이 구입한 아이템의 ID는 Int64Index([178], dtype='int64', name='Item ID')


In [63]:
# 매출 상위 Item IDs
df_grouped_by_id.sort_values('SN', ascending=False).index[1:2]


Int64Index([145], dtype='int64', name='Item ID')

In [64]:
### 이곳에서 본인 데이터로 과제를 진행해 주세요 ### 

## 🔥 도전과제

### 3) 조금 부족하더라도 최소한의 랭글링으로 데이터를 준비하여 간단한 모델을 학습시켜 보세요
- 여러분들이 선택한 데이터를 가지고 간단한 모델을 학습하고 결과를 리포팅 합니다.

In [65]:
final_test = merged_df[merged_df['status'] != 'Released']
train = merged_df[merged_df['status'] == 'Released']
train = train.drop(train[train['vote_average'] == 0].index)

In [66]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(train, train_size=0.8, test_size=0.2, random_state=2)
train, val = train_test_split(train, train_size=0.8, test_size=0.2, random_state=2)

merged_df = merged_df.drop(['id', 'movieId', 'imdbId'], axis=1)

In [67]:
import seaborn as sns
corr = merged_df.corr()
corr['rating']
df_target_corr = corr[['rating']].sort_values('rating', ascending=False)
df_target_corr

,rating
rating,1.000000
vote_average,0.555270
revenue,0.248779
runtime,0.205381
spoken_languages,0.039419
number_of_named_crews,0.033806
number_of_popular_genres,0.031820
number_of_production_countries,0.007134
number_of_named_actors,-0.027180
number_of_production_companies,-0.046427


In [68]:
target = 'rating'
features = ['vote_average', 'runtime', 'spoken_languages', 'revenue', 'homepage', 'number_of_named_crews', 'number_of_named_actors']

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]

X_test = test[features]
y_test = test[target]

X_train.shape, X_val.shape, X_test.shape

((434, 7), (109, 7), (136, 7))

In [69]:
!pip install category_encoders

     |████████████████████████████████| 82 kB 345 kB/s 


In [70]:
from sklearn.pipeline import make_pipeline
from category_encoders import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier

pipe = make_pipeline(
    OrdinalEncoder(), 
    SimpleImputer(), 
    RandomForestClassifier(max_depth=7, random_state=10, n_jobs=-1, oob_score=True)
)

pipe.fit(X_train, y_train)
print('훈련 정확도: ', pipe.score(X_train, y_train))
print('검증 정확도', pipe.score(X_val, y_val))

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


훈련 정확도:  0.9170506912442397
검증 정확도 0.8532110091743119


In [71]:
from sklearn.metrics import f1_score

y_pred_val = pipe.predict(X_val)
print('검증 f1 score: ', f1_score(y_val, y_pred_val))

검증 f1 score:  0.7647058823529411


In [72]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
y_pred = pipe.predict(X_val)

print('검증 정확도: ', accuracy_score(y_val, y_pred_val))
print(classification_report(y_pred_val, y_val))

검증 정확도:  0.8532110091743119
              precision    recall  f1-score   support

       False       0.87      0.92      0.89        73
        True       0.81      0.72      0.76        36

    accuracy                           0.85       109
   macro avg       0.84      0.82      0.83       109
weighted avg       0.85      0.85      0.85       109

